In [1]:
from pathlib import Path

from pandas import (
    read_csv,
    to_datetime,
)

from post_processing.dataclass.data_aplose import DataAplose
from post_processing.utils.fpod_utils import resample_dpm, feeding_buzz, txt_folder, build_range, csv_folder

### Load data
DPM = Detection Positive Minutes \
FB = Feeding buzzes \
🐬 = input to modify


In [3]:
yaml_file = Path(r"resource\CPOD-FPOD_yaml.yml") #Change the file path in the yaml sheet.🐬
data_list = DataAplose.from_yaml(file=yaml_file)

fb_files = Path(r"U:\fb_CA4")  #Path to your click details folder. 🐬
d_beg_end = read_csv(r"U:\Deb_Fin_CA4.csv") #Beginning and end of recording for every phase. 🐬

tl_path = Path(r"U:\TimeLost\tl_ca4")
TimeLost = csv_folder(tl_path)

### Data metric
Here you need to choose the format in which you want to visualise the positive detections. This aims to replace the functionalities available in Chelonia's softwares.

In [4]:
frq = "h" #Determine the format in which you want to visualise your data. Use "D", "h" or "10min". 🐬

In [5]:
resamp = resample_dpm(data_list.df, frq=frq, extra_columns=["deploy.name"]) #Resample your DPMs according to the chosen frq.

### Feeding buzzes processing
Use "Dauphin", Marsouin" or "Commerson" to get different ICI processing.

In [6]:
fb_all = txt_folder(fb_files) #Read all your FB.txt files.
fb_all = feeding_buzz(fb_all, "Marsouin") #Categorize the minutes (positive or not to FB detection). 🐬

fb_all["start_datetime"] = fb_all["start_datetime"].dt.floor(frq) #Resample your FBs according to the chosen frq.
fb = fb_all.groupby("start_datetime")["Foraging"].sum().reset_index()
fb["start_datetime"] = to_datetime(fb["start_datetime"], utc=True)

dpm_fb = resamp.merge(fb, on="start_datetime", how="left") #Merge DPM and FB dataframes

In [9]:
d_0 = build_range(d_beg_end, frq) #Create a dataframe from beginning to end of every phase filled with 0s.
d_tot = d_0.merge(dpm_fb, on=["start_datetime", "deploy.name"], how="left")

### Add additional metrics columns

In [10]:
d_tot[["DPM","Foraging"]] = d_tot[["DPM","Foraging"]].fillna(0)
d_tot["FBR"] = d_tot["Foraging"] / d_tot["DPM"] #The Feeding Buzz Ratio corresponds to the proportion of FB among the recorded clicks.
d_tot["FBR"] = d_tot["FBR"].fillna(0)
d_tot["DPH"] = (d_tot["DPM"] >0).astype(int) #1 if the hour counts at least 1 DPM, else 0.

In [11]:
d_tot["DPH_fb"] = (d_tot["Foraging"] >0).astype(int)
d_tot["FBR_h"] = d_tot["DPH_fb"] / d_tot["DPH"]
d_tot["FBR_h"] = d_tot["FBR_h"].fillna(0)

### Add time columns

In [12]:
d_tot = d_tot.assign(**{attr: getattr(d_tot["start_datetime"].dt, attr.lower())
                         for attr in ['Year', 'Month', 'Day', 'Hour']})
d_tot['YM'] = d_tot["start_datetime"].dt.to_period('M').dt.to_timestamp()

C:\Users\fouinel\AppData\Local\Temp\ipykernel_11560\842014434.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  d_tot['YM'] = d_tot["start_datetime"].dt.to_period('M').dt.to_timestamp()


### Extract your processed data
This dataframe is now compatible for analyses on the next notebook and on R.

In [13]:
d_tot.to_csv(r"U:\Hours_DPM_FBUZZ_CA4.csv", index=False) #Name your file. 🐬